## Additional constraints to our new model

### Constrain to enforce that every review relationship has a decision

```bash
CREATE CONSTRAINT reviewedDecisionExists
FOR ()-[r:REVIEWED]-()
REQUIRE r.decision IS NOT NULL;

### Constraint to keep only papers with mostly positive reviews

```bash
// find all papers where more than half of its reviews say “accept”
MATCH (p:article)<-[r:REVIEWED]-()
WITH
  p,
  count(*)                                         AS totalReviews,
  sum(CASE WHEN r.decision = 'accept' THEN 1 ELSE 0 END) AS numAccepts
WHERE numAccepts > totalReviews / 2
RETURN 
  p.key         AS paper,
  numAccepts,
  totalReviews;



## Query to create new REVIEW edges in the evolved graph

```bash
// (b) create new REVIEWED edges with decision & content
UNWIND [
  { paperKey: 'journals/foo/Bar20', reviewerKey: 'alice', decision: 'accept', content: 'Nice write-up!' },
  { paperKey: 'journals/foo/Bar20', reviewerKey: 'bob',   decision: 'reject', content: 'Methods unclear.'   },
  { paperKey: 'journals/foo/Bar20', reviewerKey: 'carol', decision: 'accept', content: 'Solid evaluation.' },
  … 
] AS row
MATCH (p:article {key:row.paperKey})
MATCH (a:author  {key:row.reviewerKey})
CREATE (p)-[:REVIEWED {
    decision: row.decision,
    content:  row.content,
    reviewedOn: date()
}]->(a);